# 1. Data Preprocessing

## 1.1 Import Dependencies

In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [2]:
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./data/test_round2.csv
./data/train_round2.csv
./data/graph_round2.csv
./data/test.csv
./data/graph.csv
./data/submission_round2.csv
./data/submission.csv
./data/train.csv


## 1.2 Load Data

In [3]:
data_dir = 'data'
temporal_data_path = os.path.join(data_dir, 'train_round2.csv')
mobility_data_path = os.path.join(data_dir, 'graph_round2.csv')

temporal_data = pd.read_csv(temporal_data_path)
temporal_data.describe()
#  training data from 04/12/2020 to 11/22/2020.

,ID,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,Testing_Rate,Hospitalization_Rate
count,11250.000000,1.125000e+04,11250.000000,9419.000000,1.125000e+04,11250.000000,1.055000e+04,5047.000000,10550.000000,11250.000000,5047.000000
mean,5624.500000,9.583625e+04,3057.178311,40276.189086,5.907248e+04,1357.938709,1.161268e+06,6250.044185,3.300561,20085.845676,12.197144
std,3247.739599,1.516621e+05,5377.615328,81746.843553,1.243728e+05,1199.193935,2.031813e+06,13880.694435,2.039538,19319.956138,5.255183
min,0.000000,2.700000e+02,0.000000,0.000000,-9.890000e+02,32.771004,5.459000e+03,10.000000,0.000000,460.300152,2.302896
25%,2812.250000,1.054600e+04,273.000000,3030.500000,4.599500e+03,374.587118,1.579640e+05,595.000000,1.708176,5761.788808,8.387037
50%,5624.500000,4.175800e+04,1031.000000,10376.000000,1.579100e+04,1045.850447,4.901655e+05,2068.000000,2.802207,15172.226199,11.351161
75%,8436.750000,1.174958e+05,3387.000000,48028.000000,5.289675e+04,2046.182244,1.247765e+06,6134.500000,4.425834,27209.961598,15.396088
max,11249.000000,1.153529e+06,34319.000000,913796.000000,1.095798e+06,9537.675412,1.956515e+07,89995.000000,9.741481,134755.702292,38.501190


In [4]:
states = list(np.unique(temporal_data['Province_State']))
state_cum_temporal_data = dict.fromkeys(states, None)
state_temporal_data = dict.fromkeys(states, None)
# state_cum_temporal_scaler = defaultdict(StandardScaler)
# state_temporal_scaler = defaultdict(StandardScaler)
dropped_attr = ['Date',
                'Active',
                'ID', 
                'Province_State', 
                'Incident_Rate', 
                'Recovered', 
                'People_Tested', 
                'People_Hospitalized', 
                'Mortality_Rate', 
                'Testing_Rate', 
                'Hospitalization_Rate']

for s in states:
    df_filter = temporal_data['Province_State'] == s
    state_df = temporal_data[df_filter]
    # Daily difference data
    state_temporal_data[s] = state_df.drop(dropped_attr, 1)
    for col in state_temporal_data[s]:
        data = state_temporal_data[s][col].tolist()
        diff = [i - j for i, j in zip(data, [*[data[0]], *data[:len(data) - 1]])]
        state_temporal_data[s][col] = diff
    save_columns = state_temporal_data[s].columns
#     state_temporal_data[s] = state_temporal_scaler[s].fit_transform(state_temporal_data[s])
    state_temporal_data[s] = pd.DataFrame(state_temporal_data[s], columns=save_columns)
    # Daily cumulative data
    state_cum_temporal_data[s] = state_df.drop(dropped_attr, 1)
    save_columns = state_cum_temporal_data[s].columns
#     state_cum_temporal_data[s] = state_cum_temporal_scaler[s].fit_transform(state_cum_temporal_data[s])
    state_cum_temporal_data[s] = pd.DataFrame(state_cum_temporal_data[s], columns=save_columns)
    
print(state_temporal_data['California'])
print(state_cum_temporal_data['California'])

       Confirmed  Deaths
4              0       0
54          1136      74
104         1425      53
154         1330      93
204          991      96
...          ...     ...
11004      11755     106
11054      13134      89
11104      12576      98
11154      15685      32
11204       9089      54

[225 rows x 2 columns]
       Confirmed  Deaths
4          22795     640
54         23931     714
104        25356     767
154        26686     860
204        27677     956
...          ...     ...
11004    1064040   18453
11054    1077174   18542
11104    1089750   18640
11154    1105435   18672
11204    1114524   18726

[225 rows x 2 columns]


## Train a RNN Model with LSTM

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras import Sequential
from tensorflow.keras import Input
from tensorflow.keras.models import Model

# window_size is use n-1 days to predict nth day
window_size = 22
forecast_days = 7

# test model for current_state
current_state = 'Georgia'
data_training = state_temporal_data[current_state]

In [6]:
scaler = MinMaxScaler()
data_training = scaler.fit_transform(data_training)

In [7]:
X_train = []
y_train = []

data_training_np = np.array(data_training)

for i in range(data_training.shape[0] - window_size):
    X_train.append(data_training[i : i + window_size])
    y_train.append(data_training_np[i + window_size])
    
X_train, y_train = np.array(X_train), np.array(y_train)

In [8]:
X_train.shape

(203, 22, 2)

In [9]:
y_train.shape

(203, 2)

In [10]:
model = keras.Sequential()

model.add(LSTM(units=10, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.1))

model.add(LSTM(units=20, return_sequences=False))
model.add(Dropout(0.1))

model.add(Dense(units = 2))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 22, 10)            520       
_________________________________________________________________
dropout (Dropout)            (None, 22, 10)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 20)                2480      
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense (Dense)                (None, 2)                 42        
Total params: 3,042
Trainable params: 3,042
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(optimizer='adam', loss = 'mean_squared_logarithmic_error')
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

Train on 203 samples
Epoch 1/50
203/203 [==============================] - 12s 61ms/sample - loss: 0.0084
Epoch 2/50
203/203 [==============================] - 1s 3ms/sample - loss: 0.0064
Epoch 3/50
203/203 [==============================] - 1s 3ms/sample - loss: 0.0058
Epoch 4/50
203/203 [==============================] - 1s 3ms/sample - loss: 0.0056
Epoch 5/50
203/203 [==============================] - 1s 3ms/sample - loss: 0.0055
Epoch 6/50
203/203 [==============================] - 1s 3ms/sample - loss: 0.0045
Epoch 7/50
203/203 [==============================] - 1s 3ms/sample - loss: 0.0039
Epoch 8/50
203/203 [==============================] - 1s 3ms/sample - loss: 0.0037
Epoch 9/50
203/203 [==============================] - 1s 3ms/sample - loss: 0.0036
Epoch 10/50
203/203 [==============================] - 1s 3ms/sample - loss: 0.0036
Epoch 11/50
203/203 [==============================] - 1s 3ms/sample - loss: 0.0036
Epoch 12/50
203/203 [==============================] - 1s 4ms/

In [12]:
prediction_queue = X_train[-1:]
prediction_queue = np.array(prediction_queue)
# prediction_queue

In [13]:
# prediction_queue should take in this y_pred as the last day and pop the first day in the queue
# use model.predict(prediction_queue) with the new prediction_queue to get second day.
# need to predict for next forecast_days days.
for i in range(forecast_days):  
    y_pred = model.predict(prediction_queue)
    prediction_queue = np.append(prediction_queue, y_pred)
    prediction_queue = np.delete(prediction_queue, 0)
    prediction_queue = np.delete(prediction_queue, 0)
    prediction_queue = prediction_queue.reshape(1,int(prediction_queue.shape[0]/2),2)

In [14]:
y_case_forecast = []
y_death_forecast = []

for i in range(forecast_days):
    y_case_forecast.append(prediction_queue[0][i + window_size - forecast_days][0])
    y_death_forecast.append(prediction_queue[0][i + window_size - forecast_days][1])

In [15]:
case_scale = 1/scaler.scale_[0]
death_scale = 1/scaler.scale_[1]

In [16]:
y_case_forecast = [case_scale * i for i in y_case_forecast]
y_death_forecast = [death_scale * i for i in y_death_forecast]

In [17]:
last_day_case = state_cum_temporal_data[current_state]['Confirmed'].iloc[-1]
last_day_death = state_cum_temporal_data[current_state]['Deaths'].iloc[-1]

y_cum_case_forecast = []
y_cum_death_forecast = []

cum_case = last_day_case
cum_death = last_day_death
for case in y_case_forecast:
    cum_case += case
    y_cum_case_forecast.append(cum_case)
    
for death in y_death_forecast:
    cum_death += death
    y_cum_death_forecast.append(cum_death)

In [18]:
y_cum_case_forecast

[452587.0703679025,
 456063.4661125466,
 459573.43573272973,
 463116.4421086088,
 466302.81951126456,
 469534.4814619869,
 472808.1929969862]

In [19]:
y_cum_death_forecast

[9246.42029929161,
 9294.37745976448,
 9342.326924085617,
 9390.386852502823,
 9435.158864736557,
 9480.6154088974,
 9526.674355745316]

# 2. Train a RNN with LSTM for Every State

## 2.1 Import Dependencies

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras import Sequential
from tensorflow.keras import Input
from tensorflow.keras.models import Model

# window_size is use n-1 days to predict nth day
window_size = 22
forecast_days = 7

## 2.2 Helper Functions

In [6]:
states = state_temporal_data.keys()

def get_training_data(state):
    state_temporal_train = state_temporal_data[state]
    scaler = MinMaxScaler()
    state_temporal_train = scaler.fit_transform(state_temporal_train)
    
    X_train = []
    y_train = []

    state_temporal_train_np = np.array(state_temporal_train)

    for i in range(state_temporal_train.shape[0] - window_size):
        X_train.append(state_temporal_train[i : i + window_size])
        y_train.append(state_temporal_train_np[i + window_size])

    return np.array(X_train), np.array(y_train), scaler
    

def get_model(X_train):
    model = keras.Sequential()
    
    model.add(LSTM(units = 60, activation = 'relu', return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 120, activation = 'relu', return_sequences = True))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 200, activation = 'relu'))
    model.add(Dropout(0.2))

    model.add(Dense(units = 1))
    
    return model


def get_prediction_queue(X_train):
    prediction_queue = X_train[-1:]
    return np.array(prediction_queue)


def predict(model, queue, scaler):
    prediction_queue = queue
    
    for i in range(forecast_days):  
        y_pred = model.predict(prediction_queue)
        prediction_queue = np.append(prediction_queue, y_pred)
        prediction_queue = np.delete(prediction_queue, 0)
        prediction_queue = np.delete(prediction_queue, 0)
        prediction_queue = prediction_queue.reshape(1, prediction_queue.shape[0] // 2, 2)
        
    y_case_forecast = []
    y_death_forecast = []

    for i in range(forecast_days):
        y_case_forecast.append(prediction_queue[0][i + window_size - forecast_days][0])
        y_death_forecast.append(prediction_queue[0][i + window_size - forecast_days][1])
        
    case_scale = 1 / scaler.scale_[0]
    death_scale = 1 / scaler.scale_[1]
    
    return [case_scale * i for i in y_case_forecast], [death_scale * i for i in y_death_forecast]


def get_cum_forecast(y_case_forecast, y_death_forecast):
    last_day_case = state_cum_temporal_data[current_state]['Confirmed'].iloc[-1]
    last_day_death = state_cum_temporal_data[current_state]['Deaths'].iloc[-1]

    y_cum_case_forecast = []
    y_cum_death_forecast = []

    cum_case = last_day_case
    cum_death = last_day_death
    for case in y_case_forecast:
        cum_case += case
        y_cum_case_forecast.append(cum_case)

    for death in y_death_forecast:
        cum_death += death
        y_cum_death_forecast.append(cum_death)
        
    return y_cum_case_forecast, y_cum_death_forecast

## 2.3 Train and Predict

In [7]:
forecasted = {}

for state in states:
    X_train, y_train, scaler = get_training_data(state)

    model = get_model(X_train)
    model.compile(optimizer='adam', loss = 'mean_squared_logarithmic_error')
    model.fit(X_train, y_train, epochs=50, batch_size=32)
    
    y_case_forecast, y_death_forecast = predict(model, get_prediction_queue(X_train), scaler)
    y_cum_case_forecast, y_cum_death_forecast = get_cum_forecast(y_case_forecast, y_death_forecast)
    
    forecasted[state] = (y_cum_case_forecast, y_cum_death_forecast)

Epoch 1/50
7/7 [==============================] - 0s 33ms/step - loss: 0.0556
Epoch 2/50
7/7 [==============================] - 0s 28ms/step - loss: 0.0182
Epoch 3/50
7/7 [==============================] - 0s 27ms/step - loss: 0.0163
Epoch 4/50
7/7 [==============================] - 0s 28ms/step - loss: 0.0134
Epoch 5/50
7/7 [==============================] - 0s 28ms/step - loss: 0.0128
Epoch 6/50
7/7 [==============================] - 0s 27ms/step - loss: 0.0126
Epoch 7/50
7/7 [==============================] - 0s 28ms/step - loss: 0.0119
Epoch 8/50
7/7 [==============================] - 0s 27ms/step - loss: 0.0116
Epoch 9/50
7/7 [==============================] - 0s 28ms/step - loss: 0.0113
Epoch 10/50
7/7 [==============================] - 0s 27ms/step - loss: 0.0110
Epoch 11/50
7/7 [==============================] - 0s 27ms/step - loss: 0.0110
Epoch 12/50
7/7 [==============================] - 0s 27ms/step - loss: 0.0108
Epoch 13/50
7/7 [==============================] - 0s 31ms/st

ValueError: cannot reshape array of size 43 into shape (1,21,2)